# Get Events and Stations
1. Unpickle the Model Bounding Box
1. Unpickle Groningen Shape-File 
1. Get Groningen Events
1. Plot overlay of events on the Groningen Field within Bounding Box
1. Get station data (traces) for each event
1. Plot sequence of plots with event and stations overlaid on Groningen Field

#### 1. Unpickle Bounding Box

In [ ]:
import pickle

f = open('../pickled/model_bbox.pickle', 'rb')
gf_bbox = pickle.load(f)
f.close()
print('bbox:\n',gf_bbox)

#### 2. Unpickle Groningen Shape-File

In [ ]:
import shapefile as sf

mysf = sf.Reader('../data/GroningenFieldShapeFile/Groningen_field')
print('mysf:',mysf)

#### 3. Get Groningen Events

In [ ]:
# If events are already pickled, then it is quicker to unpickle
use_pickled_events = True 

# if one wants to pickle the events again
pickle_events = False  #Think carefully before changing this!!!

In [ ]:
from gnam.events.gevents import gevents as gevents

if use_pickled_events:
    print('Unpickling Events')
    f = open('../pickled/events.pickle', 'rb')
    events = pickle.load(f)
    f.close()
else:
    print('Getting Events via FDSN')
    min_magnitude = 2.0
    events = gevents(min_magnitude,gf_bbox)
    
print()
print(events)

if pickle_events:
    print('Pickling Events')
    f = open('../pickled/events.pickle', 'wb')
    pickle.dump(events, f)
    f.close()
    

#### 4. Plot overlay of events on the Groningen Field within Bounding Box

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

#get coordinates for the Shape-File
s = mysf.shape(0)
shape_xy = np.asarray(s.points)

#get coordinates for events withing the Bounding Box
event_coords = events.getIncCoords()

#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
ax.plot(gf_bbox.getCLoop()[:,0],gf_bbox.getCLoop()[:,1],c='green',zorder=1)
ax.scatter(event_coords[:,0],event_coords[:,1],s=80,c='red',marker='*',zorder=2)
title_str = '%d Events' %(event_coords.shape[0])
ax.set_title(title_str)
plt.show()



#### 5. Get station data (traces) for each event

In [ ]:
# If stations are already pickled, then it is quicker to unpickle
use_pickled_station_traces = True  #Think carefully before changing this!!!

# if one wants to pickle the events again
pickle_station_traces = False  #Think carefully before changing this!!!

In [ ]:
from gnam.events.gstations import gstations as gstations

if use_pickled_station_traces:
    print('Unpickling Station Traces')
    f = open('../pickled/straces.pickle', 'rb')
    straces = pickle.load(f)
    f.close()
else:
    print('Getting Station Traces via Events')
    straces = gstations(events,tend=16.384)
    
print('Done')
    

if pickle_station_traces:
    print('Pickling Station Traces')
    f = open('../pickled/straces.pickle', 'wb')
    pickle.dump(straces, f)
    f.close()
    

#### 6. Plot sequence of plots with event and stations overlaid on Groningen Field

In [ ]:
# This is kind of hokey, but it works for now.
# Some of the stations depths do not follow the 
# 50, 100, 150, 200 meter depths -- possibly because
# the boreholes are slanted. To correct for this,
# a hard coded "patch/update" is applied. See the
# code for details and update values.
from gnam.events.munge.knmi import correct_station_depths as csd_f
straces.correct_stations(csd_f)


#get event and borhole keys used for indexing
ekeys = straces.getEventKeys()
bkeys = straces.getBoreholeKeys()

#Plot seuence of events with stations 
for ie in ekeys:
    # coordinates for stations that are in the bounding box
    xy3 = straces.getIncStationCoords(ie,bkeys[0]) #station code G##3
    xy4 = straces.getIncStationCoords(ie,bkeys[0]) #station code G##4
    
    # coordinates for stations that are G-stations but outside the bounding box
    ex_xy3 = straces.getExcStationCoords(ie,bkeys[0]) #station code G##3
    ex_xy4 = straces.getExcStationCoords(ie,bkeys[1]) #station code G##4
    
    # coordinates for stations that are inside the bounding box but there is no data
    er_xy3 = straces.getErrStationCoords(ie,bkeys[0]) #station code G##3
    er_xy4 = straces.getErrStationCoords(ie,bkeys[1]) #station code G##4

    fig, ax = plt.subplots(1,figsize=(8,8))
    
    #Groningen Field Shape-File
    ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
    
    #Bounding Box
    ax.plot(gf_bbox.getCLoop()[:,0],gf_bbox.getCLoop()[:,1],c='green',zorder=1)
    
    #Events (reuse event coordinates from cell further above)
    ax.scatter(event_coords[ie,0],event_coords[ie,1],s=90,c='red',marker='*',zorder=5)
    
    #Included stations
    ax.scatter(xy3[:,0],xy3[:,1],s=50,c='blue',marker='v',zorder=3)
    ax.scatter(xy4[:,0],xy4[:,1],s=100,c='gray',marker='o',zorder=2)
    
    #Excluded stations
    ax.scatter(ex_xy3[:,0],ex_xy3[:,1],s=80,c='lightgray',marker='1',zorder=4)
    ax.scatter(ex_xy4[:,0],ex_xy4[:,1],s=100,c='lightgray',marker='2',zorder=3)
    
    #Stations without data
    ax.scatter(er_xy3[:,0],er_xy3[:,1],s=50,c='yellow',marker='v',zorder=4)
    ax.scatter(er_xy4[:,0],er_xy4[:,1],s=100,c='gray',marker='o',zorder=3)
    
    origin_time = events[ie].origins[0].time
    mag = events[ie].magnitudes[0].mag
    title_str = 'Event-%d, Origin Time: %s, Magnitude: %1.2f' %(ie,str(origin_time),mag)
    ax.set_title(title_str)
    plt.show()

## Finished